In [17]:
import pandas as pd
import requests
import time
import json

In [18]:
df = pd.read_csv("mobygames_iphone-ipad-edu_2025-06-09.csv")
df.drop(columns=["moby_url"], inplace=True)

In [24]:
def safe_request(url, params, retries=3, delay=1.0):
    for attempt in range(retries):
        try:
            response = requests.get(url, params=params, timeout=10)
            if response.status_code != 200 or not response.content:
                print(f"❌ Bad response [{response.status_code}] on attempt {attempt+1}")
                time.sleep(delay)
                continue
            return response.json()
        except (requests.exceptions.RequestException, json.JSONDecodeError) as e:
            print(f"⚠️ Error on attempt {attempt+1}: {e}")
            time.sleep(delay)
    return None

In [ ]:
# Replace 'game_name_column' with the actual column name containing game names
game_names = df['title'].dropna().unique()

results = []

for name in game_names:
    params = {
        'term': name,
        'entity': 'software',
        'country': 'us',
        'limit': 1
    }
    
    data = safe_request('https://itunes.apple.com/search', params=params)
    if not data:
        continue

    if data['resultCount'] > 0:
        print("Found:", name)

        app = data['results'][0]

        result = {
            'game_name': name,
            'found': True,
            'app_store_name': app.get('trackName'),
            'average_user_rating': app.get('averageUserRating'),
            'user_rating_count': app.get('userRatingCount')
        }
        
        with open(f"midscrape/{result['game_name']}_temp.json", "w") as temp_out:
            json.dump(app, temp_out, indent=2)
            print(f"💾 Interim save: {len(result)}")

    else:
        print("No results found for:", name)
        result = {
            'game_name': name,
            'found': False,
            'app_store_name': None,
            'average_user_rating': None,
            'user_rating_count': None
        }
    
    results.append(result)
    time.sleep(0.5)  # Be polite to the API

results_df = pd.DataFrame(results)
results_df.to_csv("appstore_results.csv", index=False)

No results found for: Freddi Fish 5: The Case of the Creature of Coral Cove
No results found for: Pajama Sam 2: Thunder and Lightning aren't so Frightening
⚠️ Error on attempt 1: HTTPSConnectionPool(host='itunes.apple.com', port=443): Read timed out. (read timeout=10)
No results found for: Freddi Fish 2: The Case of the Haunted Schoolhouse
No results found for: Freddi Fish 4: The Case of the Hogfish Rustlers of Briny Gulch
No results found for: Pajama Sam: No Need to Hide When It's Dark Outside
Found: Spy Fox in "Dry Cereal"
💾 Interim save: 5
Found: Putt-Putt Saves the Zoo
💾 Interim save: 5
Found: Putt-Putt Joins the Circus
💾 Interim save: 5
Found: Putt-Putt Goes to the Moon
💾 Interim save: 5
⚠️ Error on attempt 1: HTTPSConnectionPool(host='itunes.apple.com', port=443): Read timed out. (read timeout=10)
No results found for: Spy Fox 2: "Some Assembly Required"
Found: Putt-Putt Travels Through Time
💾 Interim save: 5
No results found for: Freddi Fish 3: The Case of the Stolen Conch Shell